In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
!tar -xvf "/content/drive/MyDrive/FYP_colab/Task06_Lung.tar" -C "/content/Lung"     #[run this cell to extract tar files]

Task06_Lung/._dataset.json
Task06_Lung/dataset.json
Task06_Lung/._imagesTr
Task06_Lung/imagesTr/
Task06_Lung/imagesTr/._lung_053.nii.gz
Task06_Lung/imagesTr/lung_053.nii.gz
Task06_Lung/imagesTr/._lung_022.nii.gz
Task06_Lung/imagesTr/lung_022.nii.gz
Task06_Lung/imagesTr/._lung_041.nii.gz
Task06_Lung/imagesTr/lung_041.nii.gz
Task06_Lung/imagesTr/._lung_069.nii.gz
Task06_Lung/imagesTr/lung_069.nii.gz
Task06_Lung/imagesTr/._lung_014.nii.gz
Task06_Lung/imagesTr/lung_014.nii.gz
Task06_Lung/imagesTr/._lung_006.nii.gz
Task06_Lung/imagesTr/lung_006.nii.gz
Task06_Lung/imagesTr/._lung_065.nii.gz
Task06_Lung/imagesTr/lung_065.nii.gz
Task06_Lung/imagesTr/._lung_018.nii.gz
Task06_Lung/imagesTr/lung_018.nii.gz
Task06_Lung/imagesTr/._lung_096.nii.gz
Task06_Lung/imagesTr/lung_096.nii.gz
Task06_Lung/imagesTr/._lung_084.nii.gz
Task06_Lung/imagesTr/lung_084.nii.gz
Task06_Lung/imagesTr/._lung_086.nii.gz
Task06_Lung/imagesTr/lung_086.nii.gz
Task06_Lung/imagesTr/._lung_043.nii.gz
Task06_Lung/imagesTr/lung_04

In [ ]:
!pip install SimpleITK
!pip install monai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48.4 MB 51 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 990 kB 4.2 MB/s 


In [ ]:
import SimpleITK as sitk  #We can also use other libraries. e.g., NiBabel
from monai.transforms import CropForeground
import sys
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io 
import keras
from tensorflow.keras.layers.experimental import preprocessing
import os,cv2
from tensorflow.keras.utils import normalize

from skimage.transform import resize
from tqdm import tqdm

In [ ]:
def threshold_at_one(x):
    # threshold at 1
    return x >= 1

cropper = CropForeground(select_fn=threshold_at_one, margin=0) # Instance the Cropforground

# Generate Negative set

In [ ]:
import glob

negative_images = np.ones((1,512,512))
negative_masks = np.ones((1,512,512))

for image_path in glob.glob('/content/Lung/Task06_Lung/imagesTr/*'):
    filename = image_path[-15:-1] + image_path[-1]
    mask_path = '/content/Lung/Task06_Lung/labelsTr/' + filename

    itk_image = sitk.ReadImage(image_path)
    #Extract and save image data in numpy format
    image = sitk.GetArrayFromImage(itk_image)  #multidimensional array

    itk_mask = sitk.ReadImage(mask_path)
    #Extract and save image data in numpy format
    mask = sitk.GetArrayFromImage(itk_mask)  #multidimensional array

    # Filter the zero mask slices out
    index = np.where(np.sum(np.sum(mask, axis=1), axis = 1) == 0)
    # Random choose 3 slices in one volume
    image_number = np.random.randint(len(index[0]), size=3)
    negative_mask = mask[image_number,:,:] # (N,512,512)
    negative_image = image[image_number,:,:] # (N,512,512)

    if np.max(negative_mask) == 0:
        # Concatenate the cropped masks/imgs to the mask/img array
        negative_masks = np.concatenate((negative_masks, negative_mask), axis=0)
        negative_images = np.concatenate((negative_images, negative_image), axis=0)
        print(negative_masks.shape[0])

        if negative_masks.shape[0] > 150:
            break

negative_maskset = negative_masks[1:]
negative_imageset = negative_images[1:]

In [ ]:
negative_maskset.shape

(150, 512, 512)

In [ ]:
# Foreground cut and resize
final_images = np.ones((1,256,256))
final_masks = np.ones((1,256,256))

for i in tqdm(range(negative_imageset.shape[0])):
    ToResizei = np.reshape(negative_imageset[i], (1,512,512))
    ToResizem = np.reshape(negative_maskset[i], (1,512,512))

    # plt.figure()
    # plt.subplot(121)
    # plt.imshow(cropped_i[0], cmap='gray')
    # plt.subplot(122)
    # plt.imshow(cropped_m[0], cmap='gray')
    # plt.show()

    resized_i = resize(ToResizei, (1, 256, 256),
                        anti_aliasing=True)
    resized_m = resize(ToResizem, (1, 256, 256),
                        anti_aliasing=True)

    final_images = np.concatenate((final_images, resized_i), axis=0)
    final_masks = np.concatenate((final_masks, resized_m), axis=0)

final_images = final_images[1:]
final_masks = final_masks[1:]

100%|██████████| 150/150 [00:07<00:00, 19.72it/s]


In [ ]:
np.save('Lung_image_volume_negative', final_images)
np.save('Lung_mask_volume_negative', final_masks)

In [ ]:
np.max(final_images[0])

1321.3612983976975

# Generate SML set

In [ ]:
with open('/content/drive/MyDrive/FYP_colab/Lung_image_volume.npy', 'rb') as f:
    train_images = np.load(f)
with open('/content/drive/MyDrive/FYP_colab/Lung_mask_volume.npy', 'rb') as f:
    train_masks = np.load(f)

In [ ]:
image_dataset = np.expand_dims(train_images / np.max(train_images), 3) # only the images need normalization
mask_dataset = np.expand_dims(train_masks, 3)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, 
                                                    test_size = 0.20, random_state = 0)

In [ ]:
ratios = []
for index in range(y_test.shape[0]):
    image = y_test[index]
    ratio = np.sum(image) / 256**2
    ratios.append(ratio)

ratios = np.array(ratios)
sort_index = np.argsort(ratios) # s -> L
sorted = np.sort(ratios)

X_test_S = X_test[sort_index[:110]]
y_test_S = y_test[sort_index[:110]]

X_test_M = X_test[sort_index[111:220]]
y_test_M = y_test[sort_index[111:220]]

X_test_L = X_test[sort_index[221:]]
y_test_L = y_test[sort_index[221:]]
#     if ratio < 1e-3:
#         y_test_S = np.concatenate((y_test_S, y_test[index].reshape((1,256,256,1))), axis=0)
#         X_test_S = np.concatenate((X_test_S, y_test[index].reshape((1,256,256,1))), axis=0)

In [ ]:
np.save('Lung_L_images.npy', X_test_L)